In [1]:
# import sys
# import os

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np
from env import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from numba import njit, typed, types
from quantfreedom.nb_funcs.nb_custom_logger import *
from quantfreedom.nb_funcs.nb_helper_funcs import nb_float_to_str
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_helper_funcs import nb_get_dos
from quantfreedom.nb_funcs.nb_order_handler.nb_stop_loss import *
from quantfreedom.nb_funcs.nb_order_handler.nb_increase_position import *
from quantfreedom.nb_funcs.nb_order_handler.nb_leverage import *
from quantfreedom.nb_funcs.nb_order_handler.nb_take_profit import *
from quantfreedom.nb_funcs.nb_order_handler.nb_decrease_position import *


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=200,
)

In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

# backtest_settings = BacktestSettings(
#     array_size=1000,
#     gains_pct_filter=0,
#     total_trade_filter=40,
#     upside_filter=0,
# )

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    num_candles=50,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [5]:
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)
dynamic_order_settings = nb_get_dos(
    dos_cart_arrays=dos_cart_arrays,
    dos_index=0,
)

In [6]:
log_func_type = types.void(types.unicode_type)
logger = typed.List.empty_list(log_func_type.as_type())

str_func_type = types.unicode_type(types.float64)
stringer = typed.List.empty_list(str_func_type.as_type())

# logger.append(nb_logger_pass)
# logger.append(nb_logger_pass)
# logger.append(nb_logger_pass)
# logger.append(nb_logger_pass)

# stringer.append(nb_stringer_pass)
# stringer.append(nb_stringer_pass)
# stringer.append(nb_stringer_pass)
# stringer.append(nb_stringer_pass)
# stringer.append(nb_stringer_pass)

logger.append(nb_log_debug)
logger.append(nb_log_info)
logger.append(nb_log_warning)
logger.append(nb_log_error)

stringer.append(nb_float_to_str)
stringer.append(nb_log_datetime)
stringer.append(nb_candle_body_str)
stringer.append(nb_z_or_e_str)
stringer.append(nb_os_to_str)

In [7]:
account_state = AccountState(
    # where we are at
    ind_set_index=-1,
    dos_index=-1,
    bar_index=-1,
    timestamp=-1,
    # account info
    available_balance=static_os.starting_equity,
    cash_borrowed=0.0,
    cash_used=0.0,
    equity=static_os.starting_equity,
    fees_paid=0.0,
    possible_loss=0.0,
    realized_pnl=0.0,
    total_trades=0,
)
order_result = OrderResult(
    average_entry=0.0,
    can_move_sl_to_be=False,
    entry_price=0.0,
    entry_size_asset=0.0,
    entry_size_usd=0.0,
    exit_price=0.0,
    leverage=1.0,
    liq_price=0.0,
    order_status=OrderStatus.Nothing,
    position_size_asset=0.0,
    position_size_usd=0.0,
    sl_pct=0.0,
    sl_price=0.0,
    tp_pct=0.0,
    tp_price=0.0,
)

market_fee_pct = mufex_main.exchange_settings.market_fee_pct
leverage_tick_step = mufex_main.exchange_settings.leverage_tick_step
limit_fee_pct = mufex_main.exchange_settings.limit_fee_pct
price_tick_step = mufex_main.exchange_settings.price_tick_step
asset_tick_step = mufex_main.exchange_settings.asset_tick_step
min_asset_size = mufex_main.exchange_settings.min_asset_size
max_asset_size = mufex_main.exchange_settings.max_asset_size
max_leverage = mufex_main.exchange_settings.max_leverage
min_leverage = mufex_main.exchange_settings.min_leverage
mmr_pct = mufex_main.exchange_settings.mmr_pct

In [8]:
# stop loss
if static_os.sl_strategy_type == StopLossStrategyType.SLBasedOnCandleBody:
    nb_sl_calculator = nb_sl_based_on_candle_body
    nb_checker_sl_hit = nb_check_sl_hit
    if static_os.pg_min_max_sl_bcb.lower() == "min":
        nb_sl_bcb_price_getter = nb_min_price_getter
    elif static_os.pg_min_max_sl_bcb.lower() == "max":
        nb_sl_bcb_price_getter = nb_max_price_getter
    else:
        raise Exception("min or max are the only options for pg_min_max_sl_bcb")
else:
    nb_sl_calculator = nb_sl_calculator_pass
    nb_checker_sl_hit = nb_check_sl_hit_pass
    nb_sl_bcb_price_getter = nb_price_getter_pass

# SL break even
if static_os.sl_to_be_bool:
    nb_checker_sl_to_be = nb_check_move_sl_to_be
    # setting up stop loss be zero or entry
    if static_os.z_or_e_type.lower() == "zero":
        nb_zero_or_entry_calc = nb_sl_to_zero
    elif static_os.z_or_e_type.lower() == "entry":
        nb_zero_or_entry_calc = nb_sl_to_entry
    else:
        raise Exception("zero or entry are the only options for z_or_e_type")
else:
    nb_checker_sl_to_be = nb_cm_sl_to_be_pass
    nb_zero_or_entry_calc = nb_sl_to_z_e_pass

# Trailing stop loss
if static_os.trail_sl_bool:
    nb_checker_tsl = nb_check_move_tsl
else:
    nb_checker_tsl = nb_cm_tsl_pass

if static_os.trail_sl_bool or static_os.sl_to_be_bool:
    nb_sl_mover = nb_move_stop_loss
else:
    nb_sl_mover = nb_move_stop_loss_pass

if static_os.sl_strategy_type == StopLossStrategyType.SLBasedOnCandleBody:
    if static_os.increase_position_type == IncreasePositionType.RiskPctAccountEntrySize:
        nb_inc_pos_calculator = nb_rpa_slbcb
    elif static_os.increase_position_type == IncreasePositionType.SmalletEntrySizeAsset:
        nb_inc_pos_calculator = nb_min_asset_amount

if static_os.leverage_strategy_type == LeverageStrategyType.Dynamic:
    nb_lev_calculator = nb_dynamic_lev

nb_checker_liq_hit = nb_check_liq_hit

if static_os.tp_strategy_type == TakeProfitStrategyType.RiskReward:
    nb_tp_calculator = nb_tp_rr
    nb_checker_tp_hit = nb_c_tp_hit_regular

if static_os.tp_fee_type == TakeProfitFeeType.Market:
    exit_fee_pct = market_fee_pct
else:
    exit_fee_pct = limit_fee_pct

In [9]:
long_short = "long"
bar_index = 60

if long_short.lower() == "long":
    # stop loss
    nb_sl_to_zero_price = nb_long_sl_to_zero_price
    nb_get_sl_hit = nb_long_sl_hit_bool
    nb_move_sl_bool = nb_num_greater_than_num
    nb_sl_price_calc = nb_long_sl_price_calc

    # increase position
    nb_entry_calc_p = nb_long_entry_size_p
    nb_entry_calc_np = nb_long_entry_size_np

    # leverage
    nb_calc_dynamic_lev = nb_long_calc_dynamic_lev
    nb_get_liq_price = nb_long_get_liq_price
    nb_get_bankruptcy_price = nb_long_get_bankruptcy_price
    nb_liq_hit_bool = nb_long_liq_hit_bool

    # Take Profit
    nb_get_tp_price = nb_long_tp_price
    nb_get_check_tp_candle_price = nb_long_c_tp_candle

    # Decrease position
    nb_pnl_calc = long_pnl_calc

elif long_short.lower() == "short":
    # stop loss
    nb_sl_to_zero_price = nb_short_sl_to_zero_price
    nb_get_sl_hit = nb_short_sl_hit_bool
    nb_move_sl_bool = nb_num_less_than_num
    nb_sl_price_calc = nb_short_sl_price_calc

    # increase position
    nb_entry_calc_p = nb_short_entry_size_p
    nb_entry_calc_np = nb_short_entry_size_np

    # leverage
    nb_calc_dynamic_lev = nb_short_calc_dynamic_lev
    nb_get_liq_price = nb_short_get_liq_price
    nb_get_bankruptcy_price = nb_short_get_bankruptcy_price
    nb_liq_hit_bool = nb_short_liq_hit_bool

    # take profit
    nb_get_tp_price = nb_short_tp_price
    nb_get_check_tp_candle_price = nb_short_c_tp_candle

    # Decrease position
    nb_pnl_calc = short_pnl_calc

In [10]:
sl_price = nb_sl_calculator(
    bar_index=60,
    candles=candles,
    logger=logger,
    nb_sl_bcb_price_getter=nb_sl_bcb_price_getter,
    nb_sl_price_calc=nb_sl_price_calc,
    price_tick_step=price_tick_step,
    sl_based_on_add_pct=dynamic_order_settings.sl_based_on_add_pct,
    sl_based_on_lookback=dynamic_order_settings.sl_based_on_lookback,
    sl_bcb_type=CandleBodyType.Low,
    stringer=stringer,
)

nb_stop_loss.py - sl_bcb() - lookback to index= 40.000
nb_stop_loss.py - sl_bcb() - candle_body= 37441.300
nb_stop_loss.py - sl_bcb() - sl_price= 37422.599


In [11]:
(
    average_entry,
    entry_price,
    entry_size_asset,
    entry_size_usd,
    position_size_asset,
    position_size_usd,
    possible_loss,
    total_trades,
    sl_pct,
) = nb_inc_pos_calculator(
    acc_ex_other=AccExOther(
        equity=account_state.equity,
        asset_tick_step=asset_tick_step,
        market_fee_pct=market_fee_pct,
        max_asset_size=max_asset_size,
        min_asset_size=min_asset_size,
        possible_loss=account_state.possible_loss,
        price_tick_step=price_tick_step,
        total_trades=account_state.total_trades,
    ),
    order_info=OrderInfo(
        average_entry=order_result.average_entry,
        entry_price=candles[bar_index, CandleBodyType.Close],
        in_position=order_result.position_size_usd > 0,
        max_equity_risk_pct=dynamic_order_settings.max_equity_risk_pct,
        max_trades=dynamic_order_settings.max_trades,
        position_size_asset=order_result.position_size_asset,
        position_size_usd=order_result.position_size_usd,
        risk_account_pct_size=dynamic_order_settings.risk_account_pct_size,
        sl_price=sl_price,
    ),
    logger=logger,
    stringer=stringer,
    nb_entry_calc_p=nb_entry_calc_p,
    nb_entry_calc_np=nb_entry_calc_np,
)

nb_increase_position.py - nb_long_rpa_slbcb() - Not in a position
nb_increase_position.py - nb_rpa_slbcb_np() - Calculating
nb_increase_position.py - nb_c_pl_ra_ps() - Inside
nb_increase_position.py - nb_c_pl_ra_ps() - possible_loss= -30
nb_increase_position.py - nb_c_pl_ra_ps() - max_equity_risk= -120
nb_increase_position.py - nb_c_pl_ra_ps() - PL is fine possible_loss= -30 max risk= -120 total trades= 1
nb_increase_position.py - nb_rpa_slbcb_np() - entry_size_usd= 6072.340
nb_increase_position.py - nb_c_too_b_s() - Entry size is fineentry_size_asset= 0.162
nb_increase_position.py - nb_rpa_slbcb_np() - sl_pct= 0.300
nb_increase_position.py - nb_rpa_slbcb_np() - 
average_entry= 37540.599
entry_price= 37540.599
entry_size_asset= 0.162
entry_size_usd= 6072.340
position_size_asset= 0.162
position_size_usd= 6072.340
possible_loss= -30.000
total_trades= 1.000
sl_pct= 0.300


In [12]:
(
    available_balance,
    cash_borrowed,
    cash_used,
    leverage,
    liq_price,
) = nb_lev_calculator(
    available_balance=account_state.available_balance,
    average_entry=average_entry,
    cash_borrowed=account_state.cash_borrowed,
    cash_used=account_state.cash_used,
    leverage_tick_step=leverage_tick_step,
    logger=logger,
    market_fee_pct=market_fee_pct,
    max_leverage=max_leverage,
    min_leverage=min_leverage,
    mmr_pct=mmr_pct,
    nb_calc_dynamic_lev=nb_calc_dynamic_lev,
    nb_get_bankruptcy_price=nb_get_bankruptcy_price,
    nb_get_liq_price=nb_get_liq_price,
    position_size_asset=position_size_asset,
    position_size_usd=position_size_usd,
    price_tick_step=price_tick_step,
    sl_price=sl_price,
    static_leverage=static_os.static_leverage,
    stringer=stringer,
)

nb_leverage.py - nb_calculate_leverage() - Leverage= 122.849
nb_leverage.py - nb_calc_liq_price() -
initial_margin= 49.503
fee_to_open= 5.472
bankruptcy_price= 37235.019
cash_used= 60.405
nb_leverage.py - nb_calc_liq_price() -
available_balance= 939.594
new cash_used= 60.405
cash_borrowed= 6011.934
liq_price= 37385.199


In [13]:
(
    can_move_sl_to_be,
    tp_price,
    tp_pct,
) = nb_tp_calculator(
    average_entry=average_entry,
    logger=logger,
    market_fee_pct=market_fee_pct,
    nb_get_tp_price=nb_get_tp_price,
    position_size_usd=position_size_usd,
    possible_loss=possible_loss,
    price_tick_step=price_tick_step,
    risk_reward=dynamic_order_settings.risk_reward,
    stringer=stringer,
    tp_fee_pct=exit_fee_pct,
)

nb_take_profit.py - nb_tp_rr() - profit= 75.000
nb_take_profit.py - nb_tp_rr() - tp_price= 38049.500
nb_take_profit.py - nb_tp_rr() - tp_pct= 1.400
nb_take_profit.py - nb_tp_rr() - can_move_sl_to_be= True


In [14]:
account_state = AccountState(
    # where we are at
    ind_set_index=0,
    dos_index=0,
    bar_index=bar_index + 1,  # put plus 1 because we need to place entry on next bar
    timestamp=int(candles[bar_index + 1, CandleBodyType.Timestamp]),
    # account info
    available_balance=available_balance,
    cash_borrowed=cash_borrowed,
    cash_used=cash_used,
    equity=account_state.equity,
    fees_paid=0.0,
    possible_loss=possible_loss,
    realized_pnl=0.0,
    total_trades=total_trades,
)
order_result = OrderResult(
    average_entry=average_entry,
    can_move_sl_to_be=can_move_sl_to_be,
    entry_price=entry_price,
    entry_size_asset=entry_size_asset,
    entry_size_usd=entry_size_usd,
    exit_price=0.0,
    leverage=leverage,
    liq_price=liq_price,
    order_status=OrderStatus.EntryFilled,
    position_size_asset=position_size_asset,
    position_size_usd=position_size_usd,
    sl_pct=sl_pct,
    sl_price=sl_price,
    tp_pct=tp_pct,
    tp_price=tp_price,
)